In [1]:
%load_ext autotime

import os
import re
import io
import wget

# Define how to get data
def get_iris(storage_folder="temp", data_file="iris_data.txt", splitter=','):
    data_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
    # Make a storage folder for models and data
    if not os.path.exists(storage_folder):
        os.makedirs(storage_folder)
        
    data_path = os.path.join(storage_folder, data_file)
    
    if not os.path.isfile(os.path.join(storage_folder, data_file)):        
        _ = wget.download(data_url, data_path) 

    data = [l.strip() for l in open(data_path) if l.strip()]
    features = [tuple(map(float, x.split(splitter)[:-1])) for x in data]
    labels = [x.split(splitter)[-1] for x in data]
    
    return features, labels

def get_satellite(storage_folder="temp", data_file="satellite_data.txt", splitter=' '):
    data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/satimage/sat.trn'
    # data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/satimage/sat.tst'
    # Make a storage folder for models and data
    if not os.path.exists(storage_folder):
        os.makedirs(storage_folder)
        
    data_path = os.path.join(storage_folder, data_file)
    
    if not os.path.isfile(os.path.join(storage_folder, data_file)):           
        _ = wget.download(data_url, data_path) 

    data = [l.strip() for l in open(data_path) if l.strip()]
    features = [tuple(map(float, x.split(splitter)[:-1])) for x in data]
    labels = [x.split(splitter)[-1] for x in data]
    
    return features, labels

def get_banknote(storage_folder="temp", data_file="banknote_data.txt", splitter=','):
    data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt'
    # Make a storage folder for models and data
    if not os.path.exists(storage_folder):
        os.makedirs(storage_folder)
        
    data_path = os.path.join(storage_folder, data_file)
    
    if not os.path.isfile(os.path.join(storage_folder, data_file)):           
        _ = wget.download(data_url, data_path) 

    data = [l.strip() for l in open(data_path) if l.strip()]
    features = [tuple(map(float, x.split(splitter)[:-1])) for x in data]
    labels = [x.split(splitter)[-1] for x in data]
    
    return features, labels

vector_values, labels = get_iris()

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.mlab import PCA as mlabPCA
import tensorflow as tf
import math

num_clusters = len(set(labels))
num_vectors = len(vector_values)

print("Labels: ")
print(set(labels))
print("Sample size: ")
print(num_vectors)

vectors = tf.constant(vector_values, dtype = tf.half)

npArray = np.array(vector_values)
mlab_pca = mlabPCA(npArray)
users_2d = mlab_pca.project(npArray, minfrac=mlab_pca.fracs[1])

def drawClusters(assignment_values):
    data = {"x": [], "y": [], "cluster": []}
    for i in range(len(assignment_values)):
        data["x"].append(users_2d[i][0])
        data["y"].append(users_2d[i][1])
        data["cluster"].append(assignment_values[i])
    df = pd.DataFrame(data)
    sns.lmplot("x", "y", data=df, 
               fit_reg=False, size=7, 
               hue="cluster", legend=False)
    plt.show()

Labels: 
{'Iris-versicolor', 'Iris-virginica', 'Iris-setosa'}
Sample size: 
150
time: 1.98 s


In [3]:
def atan2(x, y, epsilon=1.0e-12):
    """
    A hack until the tf developers implement a function that can find the angle from an x and y co-ordinate.
    :param x: 
    :param epsilon: 
    :return: 
    """
    # Add a small number to all zeros, to avoid division by zero:
    x = tf.where(tf.equal(x, 0.0), x+epsilon, x)
    y = tf.where(tf.equal(y, 0.0), y+epsilon, y)

    angle = tf.where(tf.greater(x,0.0), tf.atan(y/x), tf.zeros_like(x))
    angle = tf.where(tf.logical_and(tf.less(x,0.0),  tf.greater_equal(y,0.0)), tf.atan(y/x) + np.pi, angle)
    angle = tf.where(tf.logical_and(tf.less(x,0.0),  tf.less(y,0.0)), tf.atan(y/x) - np.pi, angle)
    angle = tf.where(tf.logical_and(tf.equal(x,0.0), tf.greater(y,0.0)), 0.5*np.pi * tf.ones_like(x), angle)
    angle = tf.where(tf.logical_and(tf.equal(x,0.0), tf.less(y,0.0)), -0.5*np.pi * tf.ones_like(x), angle)
    angle = tf.where(tf.logical_and(tf.equal(x,0.0), tf.equal(y,0.0)), tf.zeros_like(x), angle)
    return angle
    
vectors_2d = tf.subtract(users_2d, tf.reduce_mean(users_2d, 0))
angles_2d = tf.add(atan2(vectors_2d[:,1], vectors_2d[:,0]), np.pi)

seg = np.pi*2/num_clusters
init_assignments = tf.cast(tf.divide(angles_2d, seg), dtype=tf.int64)

def initialize_clusters():
    init_op = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_op)
        assignment_values = sess.run(init_assignments)
        sess.close()
    return assignment_values

time: 210 ms


In [4]:
#iteration variable
assignments = tf.Variable(initialize_clusters(), dtype=tf.int64)

#clusters are based on assignments
clusters = [tf.gather(vectors, tf.squeeze(tf.where(tf.equal(assignments, c)), squeeze_dims=[1])) for c in range(num_clusters)]

time: 632 ms


In [5]:
def crossDistance(a, b, ord='euclidean'):
    diff = tf.subtract(tf.expand_dims(a, 0), tf.expand_dims(b, 1))
    result = tf.norm(diff, ord, 2)
    return result

time: 2.01 ms


In [6]:
%%time
means = tf.concat([
    tf.expand_dims(tf.reduce_mean(cluster, 0), 0) 
    for cluster in clusters], 0)
kmeans = tf.argmin(crossDistance(vectors, means), 0)

Wall time: 24 ms
time: 32 ms


In [7]:
%%time
medoids = tf.concat([
    tf.expand_dims(tf.gather(cluster, tf.argmin(tf.reduce_sum(crossDistance(cluster, cluster), 0), 0)), 0)
    for cluster in clusters], 0)
kmedoids = tf.argmin(crossDistance(vectors, medoids), 0)

Wall time: 71.1 ms
time: 74.1 ms


In [8]:
%%time
def subsample(D, size, t):
    size_D = tf.shape(D)[0]
    selector_1 = tf.range(size_D, dtype=tf.int32)
    selector_t = tf.slice(tf.map_fn(
      lambda one: tf.random_shuffle(selector_1)  
    , tf.ones([t, size_D], dtype=tf.int32)), [0,0], [t, size])
    
    return tf.gather(D, selector_t)

def HM(X, D, t=500, psi = 20, lmbda=1.0, dt=tf.half):
    size_X = tf.shape(X)[0]
    dims_X = tf.shape(X)[1]
    size_D = tf.shape(D)[0]
    
    ones_tX = tf.ones([t, size_X, dims_X], dtype=dt)  
    
    psi = tf.cond(tf.logical_and(tf.less(0, psi), tf.less(psi, size_D)), lambda: tf.constant(psi), lambda: size_D)
    set_t_psi = tf.cond(psi<size_D, 
                        lambda: subsample(D, psi, t), 
                        # order of D will not matter and it will be broadcasted
                        lambda: tf.expand_dims(D, 0)) 
    
    ones_t_psi = tf.ones([t, psi, dims_X], dtype=dt) 
    
    hm_X = tf.zeros([size_X], dtype=dt)
    
    directions = tf.random_uniform([t, dims_X], minval = -1, maxval = 1, dtype = dt)
    projectors = tf.divide(directions, tf.norm(directions, axis=1, keep_dims=True))
    
    projects_X = tf.squeeze(tf.matmul(
        tf.expand_dims(tf.multiply(ones_tX, tf.expand_dims(X, 0)), 2), 
        tf.expand_dims(tf.multiply(ones_tX, tf.expand_dims(projectors, 1)), 3)), [2, 3])
    
    projects_psi = tf.squeeze(tf.matmul(
        tf.expand_dims(tf.multiply(ones_t_psi, set_t_psi), 2), 
        tf.expand_dims(tf.multiply(ones_t_psi, tf.expand_dims(projectors, 1)), 3)), [2, 3])
    
    mid_t_min = tf.add(
        tf.multiply(tf.reduce_max(projects_psi, 1), (1-lmbda)/2),
        tf.multiply(tf.reduce_min(projects_psi, 1), (1+lmbda)/2))
    
    mid_t_max = tf.add(
        tf.multiply(tf.reduce_max(projects_psi, 1), (1+lmbda)/2),
        tf.multiply(tf.reduce_min(projects_psi, 1), (1-lmbda)/2))
    
    mid_t = tf.add(mid_t_min, tf.multiply(tf.random_uniform([t], 0, 1, dtype=dt), tf.subtract(mid_t_max, mid_t_min)))
    
    mass_l_t = tf.divide(tf.reduce_sum(
        tf.where(
            tf.less(projects_psi, tf.expand_dims(mid_t, 1)), 
            tf.ones([t, psi], dtype=dt), 
            tf.zeros([t, psi], dtype=dt)), 1), tf.cast(psi, dtype=dt))
    
    mass_r_t = tf.add(tf.multiply(mass_l_t, -1), 1)
    
    mass_t = tf.where(tf.less(projects_X, tf.expand_dims(mid_t, 1)), 
                      tf.multiply(tf.ones([t, size_X], dtype=dt), tf.expand_dims(mass_l_t, 1)), 
                      tf.multiply(tf.ones([t, size_X], dtype=dt), tf.expand_dims(mass_r_t, 1)))
    hs_mass = tf.reduce_mean(mass_t, 0)
    return hs_mass

kmass = tf.argmax(tf.concat([
    tf.expand_dims(tf.divide(HM(vectors, cluster), tf.reduce_min(HM(vectors, cluster), 0, keep_dims=True)), 1)
    for cluster in clusters
], 1), 1)

Wall time: 1.05 s
time: 1.19 s


In [9]:
%%time
%matplotlib inline

# method = kmeans
# method = kmedoids
# method = kmass

def run(method = kmeans, p=1, round_max=10, output=False):
    assignment_values = initialize_clusters()
    
    # os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
    # tf.logging.set_verbosity(tf.logging.ERROR)
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
    sess_config = tf.ConfigProto(gpu_options=gpu_options)

    init_op = tf.global_variables_initializer()
    round_i = 0
    keep_hurdle = num_vectors * p
    keep_max = -1
    best_result = assignment_values

    # drawClusters(assignment_values)
    while round_i < round_max:    
        round_i = round_i + 1
        if output:
            print(round_i)
        with tf.Session(config=sess_config) as sess:
            sess.run(init_op)
            round_result = sess.run(method)

            keep_count = np.sum([
                1 if assignment_values[i]==round_result[i] else 0
                for i in range(num_vectors)
            ])
            if output:
                print(keep_count)
            if keep_max < keep_count:
                keep_max = keep_count
                best_result = round_result
                if output:
                    print("best_result was updated.")

            assignment_values = sess.run(tf.assign(assignments, round_result))
    #         drawClusters(assignment_values)
            sess.close()
        if keep_count >= keep_hurdle:
            if output:
                print("keep_hurdle was hit.")
            break

    if output:
        print("The final P is ")
        print(float(keep_max)/float(num_vectors))    
        drawClusters(best_result)
    return best_result

Wall time: 3 ms
time: 67 ms


In [10]:
%run metrics.py

time: 6 ms


In [11]:
n_runs = 40 # number of runs with random initialization for clustering evaluation.

def evaluation_scores(groundtruth, labels_pred):
    """
    Eval scores of the predicted results.
     
    :param: groundtruth (type list): the groundtruth (GT) of cluster assignment. Each element denotes an item's GT cluster_id. 
    :param: labels_pred (type list): the predicted cluster assignments. Each element denotes an item's predicted cluster_id.
    """
    NMI = normalized_mutual_info_score(groundtruth,labels_pred)
    A = accuracy(groundtruth,labels_pred)
    F1 = f_measure(groundtruth,labels_pred)
    P = purity(groundtruth,labels_pred)
    RI = random_index(groundtruth,labels_pred)
    ARI = adjusted_rand_score(groundtruth,labels_pred)
    map_pairs = get_map_pairs(groundtruth,labels_pred)
    return NMI, A, F1, P, RI, ARI, map_pairs
    
def evaluation(method = kmeans):    
    import time
    t0 = time.time()
    NMIs,As,F1s = [],[],[]
    i_run = 1
    labels_unique = np.unique(labels)
    labels_indexed = []
    for label in labels:
        labels_indexed.append(np.where(labels_unique==label))
    labels_indexed = np.squeeze(labels_indexed)
    while i_run <= n_runs:
        t1 = time.time()
        result = run(method = method, output=False)
        NMI,A,F1,P,RI,ARI,map_pairs = evaluation_scores(labels_indexed, result)
        print("\t {}-th run(time={}s),<Acc, F1, NMI>\t{}\t{}\t{}".format(i_run, time.time()-t1, A, F1, NMI))
        i_run = i_run+1
        NMIs.append(NMI)
        As.append(A)
        F1s.append(F1)

    print("Results of {} runs (mean,std_var):\n\t Acc: {}, {}, {}, {}\n\t F1 : {}, {}, {}, {}\n\t NMI: {}, {}, {}, {}"
          .format(n_runs
                  , np.mean(As),np.std(As), np.min(As), np.max(As)
                  , np.mean(F1s),np.std(F1s), np.min(F1s), np.max(F1s)
                  ,np.mean(NMIs),np.std(NMIs), np.min(NMIs), np.max(NMIs)))
    print("Running time: {}s".format(time.time() - t0))

time: 54 ms


In [12]:
evaluation(kmass)

	 1-th run(time=8.296674251556396s),<Acc, F1, NMI>	0.7466666666666667	0.7260812328569854	0.5919190202851002
	 2-th run(time=7.836568832397461s),<Acc, F1, NMI>	0.7933333333333333	0.7776407485663176	0.6658838035484518
	 3-th run(time=7.9176554679870605s),<Acc, F1, NMI>	0.66	0.5779308453739316	0.6109504712138435
	 4-th run(time=7.928632736206055s),<Acc, F1, NMI>	0.78	0.7697215541333796	0.6048666739068796
	 5-th run(time=8.044715881347656s),<Acc, F1, NMI>	0.72	0.6881944444401107	0.580080939794489
	 6-th run(time=8.163801670074463s),<Acc, F1, NMI>	0.7666666666666667	0.7464082234605084	0.6361620165089533
	 7-th run(time=8.15660047531128s),<Acc, F1, NMI>	0.7333333333333333	0.7117084296246591	0.5675439284542232
	 8-th run(time=8.212835311889648s),<Acc, F1, NMI>	0.6666666666666666	0.5910493827124469	0.6072340071381395
	 9-th run(time=8.387960195541382s),<Acc, F1, NMI>	0.6466666666666666	0.5592954230691376	0.5943579056494029
	 10-th run(time=8.449243068695068s),<Acc, F1, NMI>	0.76	0.737276323049

In [13]:
evaluation(kmeans)

	 1-th run(time=2.0004193782806396s),<Acc, F1, NMI>	0.88	0.8787878787829131	0.7277489401358581
	 2-th run(time=1.9993066787719727s),<Acc, F1, NMI>	0.88	0.8787878787829131	0.7277489401358581
	 3-th run(time=2.0044236183166504s),<Acc, F1, NMI>	0.88	0.8787878787829131	0.7277489401358581
	 4-th run(time=2.0024654865264893s),<Acc, F1, NMI>	0.88	0.8787878787829131	0.7277489401358581
	 5-th run(time=2.026439905166626s),<Acc, F1, NMI>	0.88	0.8787878787829131	0.7277489401358581
	 6-th run(time=2.0184342861175537s),<Acc, F1, NMI>	0.88	0.8787878787829131	0.7277489401358581
	 7-th run(time=2.0214486122131348s),<Acc, F1, NMI>	0.88	0.8787878787829131	0.7277489401358581
	 8-th run(time=2.0805823802948s),<Acc, F1, NMI>	0.88	0.8787878787829131	0.7277489401358581
	 9-th run(time=1.952399492263794s),<Acc, F1, NMI>	0.88	0.8787878787829131	0.7277489401358581
	 10-th run(time=2.0234999656677246s),<Acc, F1, NMI>	0.88	0.8787878787829131	0.7277489401358581
	 11-th run(time=2.0374410152435303s),<Acc, F1, NMI>	0

In [14]:
evaluation(kmedoids)

	 1-th run(time=2.185377836227417s),<Acc, F1, NMI>	0.8933333333333333	0.8917748917699418	0.7582057278194196
	 2-th run(time=2.179297924041748s),<Acc, F1, NMI>	0.8933333333333333	0.8917748917699418	0.7582057278194196
	 3-th run(time=2.187251567840576s),<Acc, F1, NMI>	0.8933333333333333	0.8917748917699418	0.7582057278194196
	 4-th run(time=2.19069242477417s),<Acc, F1, NMI>	0.8933333333333333	0.8917748917699418	0.7582057278194196
	 5-th run(time=2.203566312789917s),<Acc, F1, NMI>	0.8933333333333333	0.8917748917699418	0.7582057278194196
	 6-th run(time=2.1850576400756836s),<Acc, F1, NMI>	0.8933333333333333	0.8917748917699418	0.7582057278194196
	 7-th run(time=2.1959967613220215s),<Acc, F1, NMI>	0.8933333333333333	0.8917748917699418	0.7582057278194196
	 8-th run(time=2.211610794067383s),<Acc, F1, NMI>	0.8933333333333333	0.8917748917699418	0.7582057278194196
	 9-th run(time=2.1985926628112793s),<Acc, F1, NMI>	0.8933333333333333	0.8917748917699418	0.7582057278194196
	 10-th run(time=2.2045938